In [ ]:
from collections import Counter, defaultdict, deque
from copy import copy
from dataclasses import dataclass

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

plt.rcParams["figure.figsize"] = [17, 17]

In [ ]:
major = [1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1]
melodic_minor = [1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1]
harmonic_minor = [1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1]
harmonic_major = [1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1]
wholetone = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]  # has only 2 colors (transpositions)
octatonic = [1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1]  # 3 colors
augmented = [1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0]  # 4 colors

scales = {
    "major": major,
    "melodic_minor": melodic_minor,
    "harmonic_major": harmonic_major,
    "harmonic_minor": harmonic_minor,
    "wholetone": wholetone,
    "octatonic": octatonic,
    "augmented": augmented,
}

In [ ]:
def contains(this, other):
    return all([(a and b) or not a for a, b in zip(other, this)])

In [ ]:
class Necklace(deque):
    def __eq__(self, other):
        assert len(self) == len(other)
        other = copy(other)
        for idx in range(len(self)):
            if deque.__eq__(self, other):
                return True
            other.rotate()
        return False

    def __ge__(self, other):
        assert len(self) == len(other)
        other = copy(other)
        for idx in range(len(self)):
            if contains(self, other):
                return True
            other.rotate()
        return False

    def __hash__(self):
        cache = [self.copy() for _ in range(len(self))]
        for n, val in enumerate(cache):
            val.rotate(n)
            cache[n] = tuple(val)

        return hash(frozenset(Counter(tuple(cache)).items()))

In [ ]:
def harp_diff(pedal_a, pedal_b):
    return sum([abs(a - b) for a, b in zip(pedal_a, pedal_b)])


natural_indices = [0, 2, 4, 5, 7, 9, 11]


def harp_add(pedal_a, pedal_b):
    return [a + b for a, b in zip(pedal_a, pedal_b)]


def harp_to_scale(pedal):
    indices = [(p + ni) % 12 for p, ni in zip(pedal, natural_indices)]
    scale = [0] * 12
    for i in indices:
        scale[i] = 1
    return scale

In [ ]:
harp_to_scale([0, -1, 0, 0, 0, 0, 0])

In [ ]:
harp_diff([0, -1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 1])

In [ ]:
from itertools import product

In [ ]:
all_pedal_positions = product(*[[-1, 0, 1]] * 7)

In [ ]:
scale_necklaces = [Necklace(s) for s in scales.values()]

In [ ]:
valid_positions = [
    pp
    for pp in all_pedal_positions
    if any([s >= Necklace(harp_to_scale(pp)) for s in scale_necklaces])
]

In [ ]:
harp_chords = defaultdict(list)

In [ ]:
for vp in valid_positions:
    harp_chords[sum(harp_to_scale(vp))].append(Necklace(harp_to_scale(vp)))

In [ ]:
for k, v in harp_chords.items():
    print(k, len(v))

In [ ]:
for k, v in harp_chords.items():
    print(k, len(set(v)))

In [ ]:
def get_children(scale):
    index = -1
    children = []
    for idx in range(sum(scale)):
        index = scale.index(1, index + 1)
        current = copy(scale)
        current[index] = 0
        if current not in children:
            children.append(current)

    return children

In [ ]:
def get_grandchildren(children):
    grandchildren = []
    for child in children:
        for grandchild in get_children(child):
            if grandchild not in grandchildren:
                grandchildren.append(grandchild)
    return grandchildren

In [ ]:
M7 = Necklace(augmented)

In [ ]:
print(M7)

In [ ]:
M6s = get_children(M7)
print(len(M6s))
for val in M6s:
    print(val)

In [ ]:
M5s = get_grandchildren(M6s)
print(len(M5s))
for val in M5s:
    print(val)

In [ ]:
M4s = get_grandchildren(M5s)
print(len(M4s))
for val in M4s:
    print(val)

In [ ]:
M3s = get_grandchildren(M4s)
print(len(M3s))
for val in M3s:
    print(val)

In [ ]:
M2s = get_grandchildren(M3s)
print(len(M2s))
for val in M2s:
    print(val)

In [ ]:
M1s = get_grandchildren(M2s)
print(len(M1s))
for val in M1s:
    print(val)

In [ ]:
chords = []

for key, value in scales.items():
    print(key)
    print("==================")
    print(sum(value), 1)
    print(Necklace(value))
    print("------------")
    chords.append(Necklace(value))

    children = get_children(Necklace(value))
    while (n_pitches := sum(children[0])) > 1:
        print(n_pitches, len(children))
        for val in children:
            if val not in chords:
                chords.append(val)
            print(val)
        print("------------")
        children = get_grandchildren(children)

print(len(chords))

In [ ]:
n_chords = defaultdict(list)

In [ ]:
for chord in chords:
    n_chords[sum(chord)].append(chord)

In [ ]:
for n_voices, chords in n_chords.items():
    print(n_voices, len(chords))
    print("--------------")
    for c in chords:
        print(c)

In [ ]:
G = nx.DiGraph()

for key, value in scales.items():
    root = Necklace(value)
    children = get_children(Necklace(value))
    G.add_node(root, subset=sum(root))

    for c in children:
        G.add_node(c, subset=sum(c))
        G.add_edge(root, c)

    while sum(children[0]) > 2:
        for c in children:
            gcs = get_children(c)
            for gc in gcs:
                G.add_node(gc, subset=sum(gc))
                G.add_edge(c, gc)
        children = get_grandchildren(children)

In [ ]:
plt.rcParams["figure.figsize"] = [8, 8]
pos = nx.multipartite_layout(G, scale=1, align="horizontal")
nx.draw_networkx_nodes(G, pos, node_size=50)
# node_labels = {node: f'{node_names[(node.a, node.b)]}{node.c}' for node in G.nodes}
# nx.draw_networkx_labels(G, {key: (val[0], val[1]+0.5) for key, val in pos.items()}, node_labels, font_size=21)
nx.draw_networkx_edges(
    G,
    pos,
    width=1,
    alpha=0.5,
)
plt.show()

In [ ]:
nodes = [(m, sum(n), n) for n, m in G.out_degree(G.nodes)]
nodes.sort(key=lambda x: G.out_degree(G.nodes)[x[2]])
nodes

In [ ]:
nodes = [(m, sum(n), n) for n, m in G.in_degree(G.nodes)]
nodes.sort(key=lambda x: G.in_degree(G.nodes)[x[2]])
nodes

In [ ]:
def rotate(x, n):
    return x[n:] + x[:n]

In [ ]:
realized_scales = []

for scale in scales.values():
    for i in range(len(scale)):
        if (realization := rotate(scale, i)) not in realized_scales:
            realized_scales.append(realization)
print(len(realized_scales))

In [ ]:
@dataclass
class Chord:
    chord: Necklace
    n_voices: int
    n_parents: int
    n_realizations: int


my_chords = []
for node in G.nodes():
    parents = [n for n in nx.algorithms.dag.ancestors(G, node) if G.in_degree(n) == 0]
    real_parents = [scale for scale in realized_scales if contains(scale, node)]
    my_chords.append(Chord(node, sum(node), len(parents), len(real_parents)))

In [ ]:
sorted(my_chords, key=lambda x: (x.n_voices, x.n_parents, x.n_realizations))
# sorted(my_chords, key=lambda x: (x.n_realizations, x.n_voices))

- 4-voice 5-parent chords

There are 7 scale families and 57 scales

In [ ]:
print("There are:")
print(f"{len(scales)} scale families")
print(f"{len(realized_scales)} scales")
for key, val in n_chords.items():
    if key < 7:
        print(f"{len(val)} types of {key}-voice chords")

In [ ]:
for val in scales.values():
    if Necklace(val) >= Necklace([0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0]):
        print(val)

In [ ]:
def distance(x, y):
    return sum(a != b for a, b in zip(x, y))

In [ ]:
# all scales that contain b and f, and does not contain c, e, and a
# sorted by their distance to C major
distances = sorted([(distance(major, rs), rs) for rs in realized_scales])
[
    d
    for d in distances
    if d[1][0] == 0 and d[1][-1] == 1 and d[1][5] == 1 and d[1][4] == 0 and d[1][9] == 0
]

In [ ]:
# all scales that contain b and f, and does not contain c, eb, and ab
# sorted by their distance to A minor
distances = sorted([(distance(major, rs), rs) for rs in realized_scales])
[
    d
    for d in distances
    if d[1][0] == 0 and d[1][-1] == 1 and d[1][5] == 1 and d[1][3] == 0 and d[1][8] == 0
]

In [ ]:
# n-voice chords that are in an certain scale
# sorted by the number of realized scales they appear in
melodic_minor_chords = [c for c in n_chords[5] if not (Necklace(major) >= c)]
parent_chords = sorted(
    [(len([rc for rc in realized_scales if contains(rc, mc)]), mc) for mc in melodic_minor_chords]
)
parent_chords

In [ ]:
Necklace(melodic_minor) >= Necklace([0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0])